**Pedro Luiz da Costa** - 
**Rafael Libertini** - 
**Gabriel Zezze**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
from PIL import Image
import os
import time
from sklearn.ensemble import RandomForestClassifier
import subprocess
from sklearn.metrics import accuracy_score

In [ ]:
#Essa função trata a foto vetorial para ser trabalhada
def convert_to_pd(tabela):
    tabela = tabela[tabela.recognized]
    tabela['timestamp'] = pd.to_datetime(tabela.timestamp)
    tabela['drawing'] = tabela['drawing'].apply(json.loads)
    return tabela

In [ ]:
airplane = pd.read_csv('airplane.csv')
octopus = pd.read_csv('octopus.csv')
airplane = convert_to_pd(airplane)
octopus = convert_to_pd(octopus)

airplane_simp = airplane[:10000]

octopus_simp = octopus[:10000]

In [ ]:
#Essa função normaliza o index e limpa as colunas desnecessárias
def limpa_tabela(tabela):
    tabela['array'] = pd.Series()
    tabela['array'] = tabela['array'].astype(object)
    tabela['index'] = np.arange(0,len(tabela))
    tabela = tabela.set_index('index')
    tabela = tabela.drop(['countrycode', 'timestamp','key_id'], axis=1)

#Essa função converte as fotos vetorias em um png e salva em uma pasta com o nome da classe.
def convert_to_png(tabela):
    nome=tabela.word[0]
    try:
        subprocess.Popen(['rm', '-rf', 'fotos/{}'.format(tabela.word[0])])
    except:
        pass

    for i, drawing in enumerate(tabela.drawing):
        for x,y in drawing:
            plt.plot(x, -np.array(y), lw=3, color='black')
            plt.axis('off')
            try:
                plt.savefig('fotos/{}/{}{}.png'.format(nome,nome,i), dpi=8)
            except:
                os.makedirs('fotos/{}'.format(nome))
                plt.savefig('fotos/{}/{}{}.png'.format(nome,nome,i), dpi=8)
                
        plt.gcf().clear()

#Essa função le o png gerado pela função acima e cria uma coluna com o array de pixels correspondente.
def to_array(tabela):
    nome=tabela.word[0]
    
    for i in range(0,len(tabela)):
        img = Image.open('fotos/{}/{}{}.png'.format(nome,nome,i)).convert('L')
        arr = np.array(img)
        
        result = []
        [result.extend(el) for el in arr/255]
        tabela['array'][i] = result

In [ ]:
limpa_tabela(airplane_simp)
convert_to_png(airplane_simp)
to_array(airplane_simp)

In [ ]:
limpa_tabela(octopus_simp)
convert_to_png(octopus_simp)
to_array(octopus_simp)

In [ ]:
#Separando o treino do teste e embaralhando o DataFrame
train = pd.concat([airplane_simp[:5000],octopus_simp[:5000]]).sample(frac=1)
test = pd.concat([airplane_simp[5000:6000],octopus_simp[5000:6000]]).sample(frac=1)

In [ ]:
X_train = list(train['array'])
y_train = list(train['word'])

X_test = list(test['array'])
y_test = list(test['word'])

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('Acurácia: {}'.format(accuracy_score(y_test, y_pred)))

In [ ]:
#Se precisar gravar o xlsx
#writer = pd.ExcelWriter('flower5K.xlsx', engine='xlsxwriter')
#flower.to_excel(writer)
#writer.save()